<a href="https://colab.research.google.com/github/Martinfuglset/refactored-journey/blob/main/Revisjon_reformat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Setup
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth, drive
from google.auth import default
from oauth2client.client import GoogleCredentials
from openpyxl import Workbook, load_workbook
import pandas as pd
import gspread
import numpy as np
from datetime import datetime

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
creds, _ = default()
gc = gspread.authorize(creds)

folder_name = 'Revisjon_excel'

counter = 0
while counter == 0:
  string = ""
  listed = drive.ListFile(({'q': f"title contains '{folder_name}'"})).GetList()
  for file in listed:
    string += 'title {}, id {}'.format(file['title'], file['id'])
    counter = 1
  if not string:
    folder_metadata = {
      'title' : folder_name,
      'mimeType' : 'application/vnd.google-apps.folder'
      }
    folder = drive.CreateFile(folder_metadata)
    folder.Upload()

fid = listed[0]['id']

In [2]:
#@title Spreadsheet

#Access / create sheet
sheet_name = 'new_wb' #@param {type:"string"}
spreadsheet = gc.open(sheet_name)
count_sheets = len(spreadsheet.worksheets())
if count_sheets == 1:
  worksheet = spreadsheet.add_worksheet(title=f"{spreadsheet.sheet1.title} (Reformatted)", rows="1000", cols="12")
else:
  worksheet = spreadsheet.worksheet(f"{spreadsheet.sheet1.title} (Reformatted)")

#Sheet URL
sheet_url = "https://docs.google.com/spreadsheets/d/" + spreadsheet.id
disp_url = True #@param {type:"boolean"}
if disp_url == True:
  print("Sheet URL: " + sheet_url)

col = ['Bilagsnr','Bilagsdato','Mva-beløp','Bokført beløp','Tekst','År','Periode','Bilagsart','Debet bokført beløp', 'Kredit bokført beløp']

rows = spreadsheet.sheet1.get_all_values()
rows_df = pd.DataFrame(rows)
indx = rows_df[rows_df[0] == "Hovedbokskonto"].index

konto = [rows_df[0][i+2] for i in indx]
navn = [rows_df[1][i+2] for i in indx]

unsort_col = rows_df.iloc[5]

unsort_index = []
for i in enumerate(unsort_col):
  try:
    unsort_index.append(col.index(i[1]))
  except:
    0

starts = []
for i in indx:
  starts.append(i+6)

ends = []
for i in enumerate(indx[:-1]):
  ends.append(indx[i[0]+1]-2)
ends.append(len(rows)-1)

bars = []
for i in enumerate(indx):
  bars.append(rows[starts[i[0]]:ends[i[0]]+1])

df = pd.DataFrame(columns=unsort_col)

konto_r = []
navn_r = []

for i in enumerate(bars):
  new = pd.DataFrame(data=bars[i[0]], columns=unsort_col)
  df = pd.concat([df,new])
  konto_r.append([konto[i[0]]]*len(new))
  navn_r.append([navn[i[0]]]*len(new))

konto = [item for sublist in konto_r for item in sublist]
navn = [item for sublist in navn_r for item in sublist]

df = df.reindex(columns=col)
df.insert(0,"Navn",navn)
df.insert(0,"Kontonr",konto)

df['Kontonr'] = df['Kontonr'].astype(int)
df['Bilagsnr'] = df['Bilagsnr'].astype(int)
df['Mva-beløp'] = df['Mva-beløp'].astype(float)
df['Bokført beløp'] = df['Bokført beløp'].astype(float)
df['År'] = df['År'].astype(int)
df['Periode'] = df['Periode'].astype(int)
df['Bilagsart'] = df['Bilagsart'].astype(int)


# con_date = []
# for i in df['Bilagsdato']:
#   dl = i.split("/")
#   if len(dl)>1:
#     con_date.append(datetime(int(dl[2]), int(dl[0]), int(dl[1])))
#   else:
#     i

# df['Bilagsdato'] = pd.DataFrame(con_date)


df['Debet bokført beløp'] = df['Bokført beløp'].where(df['Bokført beløp'] > 0, 0)
df['Kredit bokført beløp'] = df['Bokført beløp'].where(df['Bokført beløp'] < 0, 0)

df = df[df['Bilagsnr'] != 0]

worksheet.clear()
worksheet.append_row(df.columns.tolist())
worksheet.update([df.columns.values.tolist()] + df.values.tolist())

worksheet.format("A1:L1", {
    "textFormat": {
      "bold": True
    }
})


Sheet URL: https://docs.google.com/spreadsheets/d/1m91GRktQeerlSOxCxWA5yebaIuUEf-UEBjVMdXj-arU


{'spreadsheetId': '1m91GRktQeerlSOxCxWA5yebaIuUEf-UEBjVMdXj-arU',
 'replies': [{}]}